In [4]:
import sys
import pandas as pd
import json
import os

import time
import dateutil.parser
import datetime
import gc
import requests
import numpy as np


sys.path.insert(0, '../cli_test\API-V2-Python-examples')
import TAHMO

In [ ]:
'''
TAHMODATA:
    WEATHER_DATA: https://datahub.tahmo.org/services/measurements/v2/stations    
    WEATHER_DATA_RAW: measurements/raw   
    STATION_INFO: https://datahub.tahmo.org/services/assets/v2/stations
    DATA_COMPLETE: https://datahub.tahmo.org/custom/sensordx/latestmeasurements
    STATION_STATUS: https://datahub.tahmo.org/custom/stations/status   # this is new
    LAST_MEASUREMENT_OFFSET: 55
    USE_ONLY_ONE_MEASUREMENT: false
    ALWAYS_RETURN_TRUE: true
    STATIONS VARIABLES: https://datahub.tahmo.org/services/assets/v2/variables
'''
'''
TAHMOQC:
    BASE_URL: https://tahmorqctest.eu-de.mybluemix.net  # test https://tahmorqctest.eu-de.mybluemix.net
    SCORE_SUFFIX: /api/score
    ALL_MODELS_SUFFIX: /api/models
    SINGLE_MODEL_SUFFIX: /api/station   # /model is for getting by the model _id, not the station name
    SECRET_USER_PW: "6GUXzKi#wvDvZ"
    SECRET_USER_NAME: "SensorDxKenya"
    SAVE_SCORE_RESULTS: true

'''

In [45]:
# Constants
API_BASE_URL = 'https://datahub.tahmo.org'
API_MAX_PERIOD = '365D'
apiKey = 'SensorDxKenya'
apiSecret = '6GUXzKi#wvDvZ'
endpoints = {'VARIABLES': 'services/assets/v2/variables', # 28 different variables
             'STATION_INFO': 'services/assets/v2/stations',
             'WEATHER_DATA': 'services/measurements/v2/stations', # Configured before requesting
             'DATA_COMPLETE': 'custom/sensordx/latestmeasurements',
             'STATION_STATUS': 'custom/stations/status'}
# Precipitation variable
'''
"created": "2018-10-25T14:12:08.885918Z"
'''

'\n"created": "2018-10-25T14:12:08.885918Z"\n'

In [9]:
def __handleApiError(apiRequest):
    json =None
    try:
        json = apiRequest.json()
    finally:
        if json and 'error' in json and 'message' in json['error']:
            print(json)
            raise Exception(json['error']['message'])
        else:
            raise Exception(f'API request failed with status code {apiRequest.status_code}')

In [11]:
def __request(endpoint, params):
    print(f'API request: {endpoint}')
    apiRequest = requests.get(f'{API_BASE_URL}/{endpoint}',
                                params=params,
                                auth=requests.auth.HTTPBasicAuth(
                                apiKey,
                                apiSecret
                            )
    )
    if apiRequest.status_code == 200:
        return apiRequest.json()
    else:
        return __handleApiError(apiRequest)

In [ ]:
var = __request(endpoints['VARIABLES'], {})
var

In [35]:
def getVariables():
    # endpoints['VARIABLES']
    response = __request(endpoints['VARIABLES'], {})
    variables = {}
    if 'data' in response and isinstance(response['data'], list):
        for element in response['data']:
            variables[element['variable']['shortcode']] = element['variable']
    return variables

In [ ]:
getVariables()

In [37]:
def getStations():
    response = __request(endpoints['STATION_INFO'], {'sort':'code'})
    stations = {}
    if 'data' in response and isinstance(response['data'], list):
        for element in response['data']:
            stations[element['code']] = element
    return stations

In [ ]:
getStations()

In [50]:
def __splitDateRange(inputStartDate, inputEndDate):
    try:
        startDate = dateutil.parser.parse(inputStartDate)
        endDate = dateutil.parser.parse(inputEndDate)
        
    except ValueError:
        raise ValueError('Invalid data parameters')
    
    # Split into intervals of 365 days
    dates = pd.date_range(start=startDate.strftime('%Y%m%d'), end=endDate.strftime('%Y%m%d'), freq=API_MAX_PERIOD)
    df = pd.DataFrame([[i, x] for i, x in
                           zip(dates, dates.shift(1) - datetime.timedelta(seconds=1))],
                          columns=['start', 'end'])

    # Set start and end date to their provided values.
    df.loc[0, 'start'] = pd.Timestamp(startDate)
    df['end'].iloc[-1] = pd.Timestamp(endDate)
    return df
        

In [53]:
__splitDateRange('2017-01-01', '2022-10-31')

,start,end
0,2017-01-01,2017-12-31 23:59:59
1,2018-01-01,2018-12-31 23:59:59
2,2019-01-01,2019-12-31 23:59:59
3,2020-01-01,2020-12-30 23:59:59
4,2020-12-31,2021-12-30 23:59:59
5,2021-12-31,2022-10-31 00:00:00
